In [1]:
from data_preparation_functions import *
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, cross_val_score
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 100)

In [4]:
# This includes all of the data in one big dataframe
df = create_df("data/epl_data.csv")
df.head(3)

,AC,AF,AR,AS,AST,AY,AwayTeam,B365A,B365D,B365H,BWA,BWD,BWH,Bb1X2,BbAH,BbAHh,BbAv<2.5,BbAv>2.5,BbAvA,BbAvAHA,BbAvAHH,BbAvD,BbAvH,BbMx<2.5,BbMx>2.5,BbMxA,BbMxAHA,BbMxAHH,BbMxD,BbMxH,BbOU,Date,Day,Div,FTAG,FTHG,FTR,HC,HF,HR,HS,HST,HTAG,HTHG,HTR,HY,HomeTeam,IWA,IWD,IWH,LBA,LBD,LBH,Month,Referee,VCA,VCD,VCH,Year,season,gameId,homeWin,awayWin,result
0,6.0,14.0,1.0,11.0,5.0,1.0,Blackburn,2.75,3.20,2.5,2.90,3.30,2.20,55.0,20.0,0.00,1.71,2.02,2.74,2.04,1.82,3.16,2.40,1.80,2.25,2.9,2.08,1.86,3.35,2.60,35.0,2005-08-13,13,E0,1.0,3.0,H,2.0,11.0,0.0,13.0,5.0,1.0,0.0,A,0.0,West Ham,2.7,3.0,2.3,2.75,3.0,2.38,8,A Wiley,2.75,3.25,2.4,2005,0506,1,1,0,home
1,8.0,16.0,0.0,13.0,6.0,2.0,Bolton,3.00,3.25,2.3,3.15,3.25,2.10,56.0,22.0,-0.25,1.70,2.01,3.05,1.84,2.01,3.16,2.20,1.87,2.20,3.4,1.92,2.10,3.30,2.40,36.0,2005-08-13,13,E0,2.0,2.0,D,7.0,14.0,0.0,3.0,2.0,2.0,2.0,D,0.0,Aston Villa,3.1,3.0,2.1,3.20,3.0,2.10,8,M Riley,3.10,3.25,2.2,2005,0506,2,0,0,draw
2,6.0,14.0,0.0,12.0,5.0,1.0,Man United,1.72,3.40,5.0,1.75,3.35,4.35,56.0,23.0,0.75,1.79,1.93,1.69,1.86,2.00,3.36,4.69,1.87,2.10,1.8,1.93,2.05,3.70,5.65,36.0,2005-08-13,13,E0,2.0,0.0,A,8.0,15.0,0.0,10.0,5.0,1.0,0.0,A,3.0,Everton,1.8,3.1,3.8,1.83,3.2,3.75,8,G Poll,1.80,3.30,4.5,2005,0506,3,0,1,away


In [3]:
# This includes only the typical soccer stats, like home corners, home shots on target 
stats = create_stats_df("data/epl_data.csv")
stats.head(3)

,gameId,HomeTeam,AwayTeam,FTHG,FTAG,HTHG,HTAG,HS,AS,HST,AST,HF,AF,HC,AC,HY,AY,HR,AR
0,1,West Ham,Blackburn,3.0,1.0,0.0,1.0,13.0,11.0,5.0,5.0,11.0,14.0,2.0,6.0,0.0,1.0,0.0,1.0
1,2,Aston Villa,Bolton,2.0,2.0,2.0,2.0,3.0,13.0,2.0,6.0,14.0,16.0,7.0,8.0,0.0,2.0,0.0,0.0
2,3,Everton,Man United,0.0,2.0,0.0,1.0,10.0,12.0,5.0,5.0,15.0,14.0,8.0,6.0,3.0,1.0,0.0,0.0


In [7]:
# This includes all of the team information for each game
team_info = create_team_info_df("data/epl_data.csv")
team_info.head(3)

,gameId,Date,season,HomeTeam,AwayTeam,FTR,HTR,Referee
0,1,2005-08-13,0506,West Ham,Blackburn,H,A,A Wiley
1,2,2005-08-13,0506,Aston Villa,Bolton,D,D,M Riley
2,3,2005-08-13,0506,Everton,Man United,A,A,G Poll


In [9]:
historic_games = create_historic_games_df("data/historic_games_pre2005.csv")
historic_games.head(3)

C:\Users\mcelduffc\Documents\GitHub\EPL_Odds_Predictor\data_preparation_functions.py:65: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  .assign(Date=lambda df: pd.to_datetime(df.Date),


,Date,HomeTeam,AwayTeam,FTHG,FTAG,gameId,season,homeWin
0,2001-08-18,Charlton,Everton,1,2,-1,20012002,0
1,2001-08-18,Derby,Blackburn,2,1,-1,20012002,1
2,2001-08-18,Leeds,Southampton,2,0,-1,20012002,1


In [12]:
all_games = create_all_games_df("data/epl_data.csv", "data/historic_games_pre2005.csv")
all_games.head(3)

C:\Users\mcelduffc\Documents\GitHub\EPL_Odds_Predictor\data_preparation_functions.py:65: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  .assign(Date=lambda df: pd.to_datetime(df.Date),
C:\Users\mcelduffc\Documents\GitHub\EPL_Odds_Predictor\data_preparation_functions.py:82: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  all_games = (historic_games.append(df[historic_games.columns])


,Date,HomeTeam,AwayTeam,FTHG,FTAG,gameId,season,homeWin,awayWin,homeWinPc5,homeWinPc38,awayWinPc5,awayWinPc38,gameIdHistoric
0,2001-08-18,Charlton,Everton,1.0,2.0,-1,20012002,0,1,NaN,NaN,NaN,NaN,1
1,2001-08-18,Derby,Blackburn,2.0,1.0,-1,20012002,1,0,NaN,NaN,NaN,NaN,2
2,2001-08-18,Leeds,Southampton,2.0,0.0,-1,20012002,1,0,NaN,NaN,NaN,NaN,3


## Feature Creation

* The data is now clean and has been separated into pre-prepared DataFrames so it is important to now focus on feature creation.
* Common practice with sports modelling to integrate the use of exponentially weighted moving averages (EMA) as features.
* An EMA is like a simple moving average, except it weights recent instances more than older instances based on alpha parameter.
* EMAs can quickly adjust to changes in a team's performance level, making them valuable for dynamic environments like sports where teams can hit hot streaks or suffer downturns in form.
* Since EMAs are recursive, you need an initial value to start the calculation. This could be the average of a set of initial games, or it could be set to the first game's value.

Define a function which calculates the exponential moving average for each column in the stats DataFrame. Then apply this function with other functions that have been created, such as create_betting_features, which creates moving averages of betting data.

May make things difficult to calculate rolling averages when having Home Team's data and Away Team's data on a single row. Will ahve tp restructure the DataFrames to ensure each row onyl contains single team's data.

In [13]:
# Function to restructure the DataFrame
def create_multiline_df_stats(old_stats_df):
    home_stats_cols = ['HomeTeam', 'FTHG', 'FTAG', 'HTHG', 'HTAG', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY',
                       'HR', 'AR']
    
    away_stats_cols = ['AwayTeam', 'FTAG', 'FTHG', 'HTAG', 'HTHG', 'AS', 'HS', 'AST', 'HST', 'AF', 'HF', 'AC', 'HC', 'AY', 'HY',
                       'AR', 'HR']
    
    stats_cols_mapping = ['team', 'goalsFor', 'goalsAgainst', 'halfTimeGoalsFor', 'halfTimeGoalsAgainst', 'shotsFor',
                          'shotsAgainst', 'shotsOnTargetFor', 'shotsOnTargetAgainst', 'freesFor', 'freesAgainst', 
                          'cornersFor', 'cornersAgainst', 'yellowsFor', 'yellowsAgainst', 'redsFor', 'redsAgainst']
    
    home_mapping = {old_col: new_col for old_col, new_col in zip(home_stats_cols, stats_cols_mapping)}
    away_mapping = {old_col: new_col for old_col, new_col in zip(away_stats_cols, stats_cols_mapping)}
    
    multi_line_stats = (old_stats_df[['gameId'] + home_stats_cols] # Filter for only the home team columns
                    .rename(columns=home_mapping) # Rename the columns
                    .assign(homeGame=1) # Assign homeGame=1 so that we can use a general function later
                    .append((old_stats_df[['gameId'] + away_stats_cols]) # Append the away team columns
                            .rename(columns=away_mapping) # Rename the away team columns
                            .assign(homeGame=0), sort=True)
                    .sort_values(by='gameId') # Sort the values
                    .reset_index(drop=True))
    return multi_line_stats

In [15]:
# Define a function which creates an EMA DataFrame from the stats DataFrame
def create_stats_features_ema(stats, span):
    # Create a restructured DataFrames so that we can calculate EMA
    multi_line_stats = create_multiline_df_stats(stats)
    
    # Create a copy of the DataFrame
    ema_features = multi_line_stats[['gameId', 'team', 'homeGame']].copy()
    
    # Get the columns that we want to create EMA for
    feature_names = multi_line_stats.drop(columns=['gameId', 'team', 'homeGame']).columns
    
    # Loop over the features
    for feature_name in feature_names:
        feature_ema = (multi_line_stats.groupby('team')[feature_name] # Calculate the EMA
                                                  .transform(lambda row: row.ewm(span=span, min_periods=2)
                                                             .mean()
                                                             .shift(1))) # Shift the data down 1 so we don't leak data
        ema_features[feature_name] = feature_ema # Add the new feature to the DataFrame
    return ema_features

In [21]:
# Apply the function
stats_features = create_stats_features_ema(stats, span=5)
stats_features.tail()

C:\Users\mcelduffc\AppData\Local\Temp\ipykernel_80604\2101142793.py:19: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  .append((old_stats_df[['gameId'] + away_stats_cols]) # Append the away team columns


,gameId,team,homeGame,cornersAgainst,cornersFor,freesAgainst,freesFor,goalsAgainst,goalsFor,halfTimeGoalsAgainst,halfTimeGoalsFor,redsAgainst,redsFor,shotsAgainst,shotsFor,shotsOnTargetAgainst,shotsOnTargetFor,yellowsAgainst,yellowsFor
9903,4952,Newcastle,1,4.301743,4.217300,11.789345,12.245066,0.797647,0.833658,0.644214,0.420832,2.323450e-10,3.333631e-01,11.335147,13.265955,3.211345,4.067990,1.848860,1.627140
9904,4953,Burnley,0,4.880132,5.165915,13.326703,8.800033,1.945502,0.667042,0.609440,0.529409,3.874405e-03,3.356120e-10,13.129631,10.642381,4.825874,3.970285,0.963527,0.847939
9905,4953,Fulham,1,4.550255,4.403060,10.188263,8.555589,2.531046,1.003553,0.860573,0.076949,1.002518e-04,8.670776e-03,17.463779,12.278877,8.334019,4.058213,0.980097,1.102974
9906,4954,Man United,1,3.832573,4.759683,11.640608,10.307946,1.397234,1.495032,1.034251,0.809280,6.683080e-05,1.320468e-05,8.963022,10.198642,3.216957,3.776900,1.040077,1.595650
9907,4954,Tottenham,0,3.042034,5.160211,8.991460,9.955635,1.332704,2.514789,0.573728,1.010491,4.522878e-08,1.354409e-05,12.543406,17.761004,3.757437,7.279845,1.478976,1.026601


We now have the averages for each team. Below is a table to see the top 10 teams' goalsFor average EMAs since 2005.

In [22]:
pd.DataFrame(stats_features.groupby('team')
                            .goalsFor
                                .mean()
                                    .sort_values(ascending=False)[:10])

,goalsFor
team,
Man United,1.895026
Chelsea,1.888892
Arsenal,1.876770
Man City,1.835863
Liverpool,1.771125
Tottenham,1.655063
Leicester,1.425309
Blackpool,1.390936
Everton,1.387110


## Optimising Alpha

* Man United and Chelsea seem to be the two of the best teams since 2005, based on goalsFor. 
* However, it might be worth checking different span values as we cannot be sure 5 is correct here.

To check this, we will use a simple Logistic Regression model to create probabilistic predictions based on the stats features we created before.<br>
<br>
Iterate through a range of span values from, 3 to 15, and choose the value which produces a model with the lowest log loss, based on cross validation

In [23]:
def restructure_stats_features(stats_features):
    non_features = ['homeGame', 'team', 'gameId']

    stats_features_restructured = (stats_features.query('homeGame == 1')
                                    .rename(columns={col: 'f_' + col + 'Home' for col in stats_features.columns if col not in non_features})
                                    .rename(columns={'team': 'HomeTeam'})
                                    .pipe(pd.merge, (stats_features.query('homeGame == 0')
                                                        .rename(columns={'team': 'AwayTeam'})
                                                        .rename(columns={col: 'f_' + col + 'Away' for col in stats_features.columns 
                                                                         if col not in non_features})), on=['gameId'])
                                    .pipe(pd.merge, df[['gameId', 'result']], on='gameId')
                                    .dropna())
    return stats_features_restructured

restructure_stats_features(stats_features).head()

,gameId,HomeTeam,homeGame_x,f_cornersAgainstHome,f_cornersForHome,f_freesAgainstHome,f_freesForHome,f_goalsAgainstHome,f_goalsForHome,f_halfTimeGoalsAgainstHome,f_halfTimeGoalsForHome,f_redsAgainstHome,f_redsForHome,f_shotsAgainstHome,f_shotsForHome,f_shotsOnTargetAgainstHome,f_shotsOnTargetForHome,f_yellowsAgainstHome,f_yellowsForHome,AwayTeam,homeGame_y,f_cornersAgainstAway,f_cornersForAway,f_freesAgainstAway,f_freesForAway,f_goalsAgainstAway,f_goalsForAway,f_halfTimeGoalsAgainstAway,f_halfTimeGoalsForAway,f_redsAgainstAway,f_redsForAway,f_shotsAgainstAway,f_shotsForAway,f_shotsOnTargetAgainstAway,f_shotsOnTargetForAway,f_yellowsAgainstAway,f_yellowsForAway,result
20,21,Birmingham,1,4.8,7.8,12.0,9.4,1.2,0.6,0.6,0.6,0.0,0.0,11.4,8.2,6.4,2.8,1.0,2.6,Middlesbrough,0,3.0,5.6,14.0,12.8,1.2,0.0,0.0,0.0,0.0,0.4,17.2,8.8,7.6,2.6,3.0,1.4,away
21,22,Portsmouth,1,2.6,4.6,21.8,16.6,2.0,0.6,1.0,0.0,0.0,0.0,8.0,10.4,3.6,4.0,3.2,1.8,Aston Villa,0,9.8,7.0,14.2,18.2,1.4,0.8,0.8,0.8,0.0,0.0,16.0,3.0,9.6,2.6,2.0,0.6,draw
22,23,Sunderland,1,5.0,5.0,11.6,18.0,1.8,0.4,1.0,0.4,0.4,0.6,14.6,6.0,5.2,3.2,1.2,2.6,Man City,0,7.8,3.6,8.6,12.4,0.6,1.2,0.6,0.6,0.0,0.0,10.6,11.4,2.4,6.8,3.0,1.4,away
23,24,Arsenal,1,3.0,7.4,17.0,18.6,0.6,0.8,0.0,0.0,0.4,0.0,6.2,11.4,4.0,6.6,1.6,1.8,Fulham,0,7.2,3.0,20.8,13.2,1.2,0.6,0.6,0.0,0.0,0.0,12.4,10.8,7.0,5.2,2.0,1.6,home
24,25,Blackburn,1,1.4,7.2,12.8,21.2,1.8,1.6,0.0,1.0,0.0,0.4,10.0,14.0,4.4,7.4,1.2,1.6,Tottenham,0,6.4,3.8,11.2,18.8,0.0,2.0,0.0,0.4,0.0,0.0,11.6,15.2,4.6,7.2,0.6,2.6,draw


Now to write a function that optimises our span based on log loss of the output of a Logistic Regression model.

In [24]:
def optimise_alpha(features):
    le = LabelEncoder()
    y = le.fit_transform(features.result) # Encode the result from away, draw, home win to 0, 1, 2
    X = features[[col for col in features.columns if col.startswith('f_')]] # Only get the features - these all start with f_
    lr = LogisticRegression()
    
    kfold = StratifiedKFold(n_splits=5)
    ave_cv_score = cross_val_score(lr, X, y, scoring='neg_log_loss', cv=kfold).mean()
    return ave_cv_score

In [26]:
# best_score = np.float('inf')
# best_span = 0
# cv_scores = []

# # Iterate over a range of spans
# for span in range(1, 120, 3):
#     stats_features = create_stats_features_ema(stats, span=span)
#     restructured_stats_features = restructure_stats_features(stats_features)
#     cv_score = optimise_alpha(restructured_stats_features)
#     cv_scores.append(cv_score)
    
#     if cv_score * -1 < best_score:
#         best_score = cv_score * -1
#         best_span = span